In [1]:
import sys
import numpy as np
import torch
import gc

# Req for package
sys.path.append("../")
from SkinLearning.NN.Models import DualDown, DualDownUp, DualUp, DualUpDown, MultiTemporal
from SkinLearning.NN.Helpers import kfcv, DEVICE, set_seed, train, test
from SkinLearning.Utils.Plotting import plot_parameter_bars, save_df
from SkinLearning.Utils.Dataset import get_dataset, get_split

In [2]:
extraction_args = {
            "signals": None,
            "method": "entropy",
            "combined": False,
            "wavelet": "db4",
            "level": 7,
            "order": "natural",
            "levels": [7],
            "normalization": False,
            "stats": None,
        }

level7, scaler7 = get_dataset(extraction_args=extraction_args)
extraction_args['level'] = 6
extraction_args['levels'] = [6]
level6, scaler6 = get_dataset(extraction_args=extraction_args)
extraction_args['level'] = 8
extraction_args['levels'] = [8]
level8, scaler8 = get_dataset(extraction_args=extraction_args)
extraction_args['level'] = 5
extraction_args['levels'] = [5]
level5, scaler5 = get_dataset(extraction_args=extraction_args)

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 209.94it/s]


In [3]:
extraction_args['level'] = 8
extraction_args['levels'] = [8]
level8, scaler8 = get_dataset(extraction_args=extraction_args)
extraction_args['level'] = 5
extraction_args['levels'] = [5]
level5, scaler5 = get_dataset(extraction_args=extraction_args)

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 207.97it/s]


In [4]:
loaders = np.array([
    get_split(level5),
    get_split(level6),
    get_split(level7),
    get_split(level8)
])

scalers = np.array([
    scaler5,
    scaler6,
    scaler7,
    scaler8
])

In [5]:
def get_models():
    return [
        MultiTemporal(
            conv=False,
            out="f_output",
            hidden_size=len(level5[0]['input'])*2,
            input_size=len(level5[0]['input']),
            temporal_type="LSTM"
        ),
        MultiTemporal(
            conv=False,
            out="f_output",
            hidden_size=len(level6[0]['input'])*2,
            input_size=len(level6[0]['input']),
            temporal_type="LSTM"
        ),
        MultiTemporal(
            conv=False,
            out="f_output",
            hidden_size=len(level7[0]['input'])*2,
            input_size=len(level7[0]['input']),
            temporal_type="LSTM"
        ),
        MultiTemporal(
            conv=False,
            out="f_output",
            hidden_size=len(level8[0]['input'])*2,
            input_size=len(level8[0]['input']),
            temporal_type="LSTM"
        ),
    ]

In [6]:
names = [5, 6, 7, 8]

In [7]:
def get_avg(models, run=1):
    for i, model in enumerate(models):
        print(f"Training with level {names[i]} for run {i}")
        train(loaders[i, 0], model, epochs=1500, early_stopping=True, val_loader=loaders[i, 1])
    
    maes = []
    for i, model in enumerate(models):
        _, _, model_mae = test(loaders[i, 1], model, scalers[i])
        maes.append(model_mae)
        
    return maes

In [8]:
def repeat_avg(repeats=5):
    maes = []
    for i in range(repeats):
        models = get_models()
        
        single_maes = get_avg(models, run=i)
        maes.append(single_maes)
        
        del models
        torch.cuda.empty_cache()
        gc.collect()
    
    mean = np.mean(np.array(maes), axis=0)
    print(mean)
    
    return mean

In [9]:
repeat_avg()

FC in: 128 HS 128
FC in: 256 HS 256
FC in: 512 HS 512
FC in: 1024 HS 1024
Training with level 5 for run 0
Using: cuda


100%|█████████████████| 113/113 [00:00<00:00, 320.12batch/s, counter=1, epoch=68/1500, lastLoss=0.0816, valLoss=0.0909]


100%|████████████████| 113/113 [00:00<00:00, 296.98batch/s, counter=16, epoch=136/1500, lastLoss=0.072, valLoss=0.0925]


100%|███████████████| 113/113 [00:00<00:00, 328.01batch/s, counter=29, epoch=204/1500, lastLoss=0.0715, valLoss=0.0891]


100%|███████████████| 113/113 [00:00<00:00, 315.65batch/s, counter=49, epoch=224/1500, lastLoss=0.0755, valLoss=0.0978]


...

100%|█████████████████| 113/113 [00:00<00:00, 218.57batch/s, counter=17, epoch=68/1500, lastLoss=0.0981, valLoss=0.116]


100%|████████████████| 113/113 [00:00<00:00, 218.15batch/s, counter=2, epoch=136/1500, lastLoss=0.0783, valLoss=0.0886]


100%|████████████████| 113/113 [00:00<00:00, 232.27batch/s, counter=49, epoch=183/1500, lastLoss=0.0712, valLoss=0.123]


Early stopping after 183 epochs
Average train loss: 0.011481938799837697
Average validation loss: 0.01404964013611791
[0.09412284 0.08016449 0.08063472 0.10636761]


array([0.09412284, 0.08016449, 0.08063472, 0.10636761])